In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

### Python code to find the sequence of entropies by slicing through segments of a given file at different positions
### Here, we treat the byte sequence as a signal (with amplitude ranging from 0 to 255) and transform it into frequency space.


In [2]:
def lempel_ziv_complexity(binary_sequence):
    sub_strings = set()
    n = len(binary_sequence)

    ind = 0
    inc = 1
    while True:
        if ind + inc > len(binary_sequence):
            break
        sub_str = binary_sequence[ind : ind + inc]
        if sub_str in sub_strings:
            inc += 1
        else:
            sub_strings.add(sub_str)
            ind += inc
            inc = 1
    return len(sub_strings)


def shannon_entropy(data_classification_sequence):
  
    
    """
    
    HTML PNG JPEG GIF PDF DOC ELF GZIP AES
    
    
    """
    
    # Hold the relative frequency per data file for a given classification sequence
    relative_freq_datafile = {
        'HTML' : 0, 
        'PNG' : 0, 
        'JPEG' : 0, 
        'GIF': 0,
        'PDF' : 0,
        'DOC' : 0,
        'ELF' : 0,
        'GZIP' : 0,
        'AES' : 0
    }
    
    
    
    # Formula looks like this
    # H(S) = −Σ P(Si) log2 (P(Si))
    # P(Si) is a bit confusing but, it's the relative frequency of the current char i in the whole string 
    # as we are iterating on each character in the string. 

    
    # step 1: calculate all the frequency for each characters
    for datafile in relative_freq_datafile:
        relative_freq_datafile[datafile] = data_classification_sequence.count(datafile) / len(data_classification_sequence)
    
    # step 2: iterate over each nucleotide and sum up the relative frequency
    entropy = 0
    for datafile_i in data_classification_sequence:
        rel_freq = relative_freq_datafile[datafile_i]
        entropy = entropy - (rel_freq * math.log(rel_freq, 2))
        
    return entropy

def shannon_entropy_corrected(data_classification_sequence):
    """Custom implementation of shannon entropy with a full non-binarized sequence
        Formula looks like this
        H(S) = −Σ P(Si) log2 (P(Si))
        P(Si) here is simply the relative frequency of character HTML,PNG,JPEG,GIF,PDF,DOC,ELF,GZIP,AES or n in the string.
    """
    
    entropy = 0
    for datafile in {'HTML','PNG','JPEG','GIF','PDF','DOC','ELF','GZIP','AES'}:
        rel_freq = data_classification_sequence.count(datafile) / len(data_classification_sequence)
        if rel_freq > 0:
            entropy = entropy - (rel_freq * math.log(rel_freq, 2))
        
    return entropy

def string_to_binary(string_sequence):
    """Simple converter from a string sequence to a binary sequence"""
    return ''.join(format(ord(x), 'b') for x in string_sequence)

def generate_window(sequence, window_size=256, jump_size=50):
    """ Generator of windowed sequence with specific jump size
    Parameters:
    sequence (str): a string representing the data
    window_size (int): the size of the window we want to return
    jump_size (int): how much we want to move the window after each yield
    
    Yield:
    windowed sequence of specific size
    
    """
    start = 0
    end = window_size
    
    while end < len(sequence):
        yield sequence[start:end]
        
        start = start + jump_size
        end = end + jump_size

def generate_lz_sequence(sequence):
    """helper function to generate a list of Lempel-Ziv complexity metric"""
    lz_sequence = []
    for window in generate_window(sequence):
        encoded_sequence = string_to_binary(window)
        lz = lempel_ziv_complexity(encoded_sequence)
        lz_sequence.append(lz)
    
    return lz_sequence

def generate_shannon_sequence(sequence, shannon_fn):
    """helper function to generate a list of Shannon Entropy metric"""
    se_sequence = []
    for window in generate_window(sequence):
        se = shannon_fn(window)
        se_sequence.append(se)
        
    return se_sequence

def normalize_lz_sequence(lz_sequence, window_size=250):
    """helper function to normalize a sequence of Lempel-Ziv complexity metric"""
    return [lz/window_size for lz in lz_sequence]

In [50]:
## Randomly generated data
np.random.seed(1)
data3 = pd.DataFrame({" HTML" : np.random.randint(low=0, high=2, size=1000),
                     "PNG"  : np.random.randint(0, 2, size=1000),
                      "JPEG"  : np.random.randint(0, 2, size=1000),
                      "GIF"  : np.random.randint(0, 2, size=1000),
                      "PDF"  : np.random.randint(0, 2, size=1000),
                      "DOC"  : np.random.randint(0, 2, size=1000),
                      "ELF"  : np.random.randint(0, 2, size=1000),
                      "GZIP"  : np.random.randint(0, 2, size=1000),
                      "AES"  : np.random.randint(0, 2, size=1000)
                     })
data3

,HTML,PNG,JPEG,GIF,PDF,DOC,ELF,GZIP,AES
0,1,0,1,1,1,0,1,0,1
1,1,0,1,1,1,0,0,1,1
2,0,0,0,0,1,0,1,0,1
3,0,1,1,0,1,1,0,1,0
4,1,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
995,0,0,0,1,0,1,0,1,0
996,0,1,0,1,1,1,1,1,0
997,0,0,0,1,0,0,1,0,0
998,0,0,0,0,1,0,1,1,1


### Preparing Data for neural network

In [3]:
#Enter the data here,
#df = pd.read_csv('#file.csv')

In [4]:
# Assuming the file has data with column 'filetypes' which contains a sequence of file types we need to classify.
datafile_list = data3
main_data_sequence = ''.join(datafile_list)
len(main_seq)



# Generating Shannon Entropy
se_seq = generate_shannon_sequence(main_data_sequence, lambda seq : shannon_entropy(seq))
len(se_seq)




# Generating Corrected Shannon Entropy
se_seq_cor = generate_shannon_sequence(main_data_sequence, lambda seq : shannon_entropy_corrected(seq))
len(se_seq_cor)




# Generating the lempel-ziv sequence with window of 256 datafile formats and jumps of 50 datafiles
lz_sequence = generate_lz_sequence(main_data_sequence)
len(lz_seq)




# Normalizing the LZ complexity between 0 and 1
n_lz_sequence = normalize_lz_sequence(lz_sequence, 256)

NameError: name 'data3' is not defined

In [ ]:
# For this project, i will generate random dummy data
x_train = np.random.random((1000, 9))
y_train = keras.utils.to_categorical(np.random.randint(9, size=(1000,1)),num_classes=9)
x_test = np.random.random((100, 9))
y_test = keras.utils.to_categorical(np.random.randint(9, size=(1000,1)),num_classes=9)

In [ ]:
plt.plot(y_train)

In [ ]:
# If we have an existing dataset,

# Splitting the dataset

from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20, random_state=42)

In [ ]:
# Model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = 9))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.09, nesterov=True)

In [ ]:
model.compile(loss='categorical_classentrepy',
             optimizer = sgd,
             metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=42)